In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

from clearml import Task, Logger

2026-01-02 16:23:19.595851: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-02 16:23:19.673167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-02 16:23:20.900186: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/id/sandbox/MAI/nn/venv/lib/python3.12/site-packages/keras/src/

### Настроим основные гиперпараметры

Обучение моделей буду дополнительно логировать в ClearML, чтобы дополнительно к лабам поиграться-поучиться/привыкнуть пользоваться им (нужно для работы)

In [2]:
# Инициализация задачи ClearML (замените 'my_project' и 'my_exp' на свои)
task = Task.init(project_name='here_we_go_again', task_name='CNN Binary Classifier - Buildings vs Forest')

# Гиперпараметры процесса обучения
EPOCHS = 10
BATCH_SIZE = 128
IMG_HEIGHT = 224  # Примерный размер, можно изменить
IMG_WIDTH = 224   # Примерный размер, можно изменить
NUM_CLASSES = 2 # binary classification


ClearML Task: overwriting (reusing) task id=f900ee3b00394376a558804600c3231a
ClearML results page: http://10.31.10.2:8080/projects/238712d43c2843d2ab9525fa5f876cd8/experiments/f900ee3b00394376a558804600c3231a/output/log
CLEARML-SERVER new package available: UPGRADE to v2.3.0 is recommended!
Release Notes:
### New Features and Improvements

- New Project Workloads dashboard: View project resource utilization by resource, user, and subproject
- New Global Search advanced mode: Direct API filter specification for more specific queries
- Improve single-value scalar Compare view: Separate plot per variant when grouping by metric
- Enable customizing the default name format of cloned tasks 

### Bug Fixes

- Fix UI GCS credentials popup continues reappearing after broken upload #296
- Fix UI sometimes fails to load large task console logs #295
- Fix deleting parameter in UI task configuration causes other parameters to be duplicated or incorrectly modified 
- Fix deleting task in UI removes 

ClearML results page: http://10.31.10.2:8080/projects/238712d43c2843d2ab9525fa5f876cd8/experiments/f900ee3b00394376a558804600c3231a/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError("HTTPConnection(host='10.31.10.2', port=8008): Failed to establish a new connection: [Errno 101] Network is unreachable")': /v2.23/events.add_batch
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError("HTTPConnection(host='10.31.10.2', port=8008): Failed to establish a new connection: [Errno 101] Network is unreachable")': /v2.23/events.add_batch
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError("HTTPConnection(host='10.31.10.2', port=8008): Failed to establish a new connection: [Errno 101] Network is unreachable")': /v2.23/events.add_batch


### Загрузим датасет и нормализуем значения пикселей

In [3]:
data_dir = "data/lab1_2_6/4.Scenes_buildings_vs_forest"

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, # 20% данных для валидации
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary' # Для бинарной классификации
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# Нормализация значений пикселей к диапазону [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Found 4462 files belonging to 2 classes.
Using 3570 files for training.
Found 4462 files belonging to 2 classes.
Using 892 files for validation.


2026-01-02 16:23:25.117132: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Создадим простенькую модель-классификатор

In [4]:
model = Sequential([
           Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # 3 канала для RGB
           Conv2D(32, (3, 3), activation='relu'),
           MaxPooling2D((2, 2)),
           Conv2D(64, (3, 3), activation='relu'),
           MaxPooling2D((2, 2)),
           Conv2D(256, (3, 3), activation='relu'),
           Flatten(),
           Dense(40, activation='relu'),
           # Изменяем последний слой на 1 нейрон с сигмоидой для бинарной классификации
           Dense(1, activation='sigmoid'), 
        ]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 256)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 692224)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │    27,689,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,856,145 (106.26 MB)

 Trainable params: 27,856,145 (106.26 MB)

 Non-trainable params: 0 (0.00 B)

### Настроим модель для обучения, дополнительно добавив механизм ранней остановки для предотввращения переобучения

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='binary_crossentropy', # Меняем функцию потерь именно для бинарной классификации
              optimizer=Adam(learning_rate=3e-4),
              metrics=['accuracy'])


# Создание колбэка
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Метрика, за которой следим
    patience=5,
    min_delta=0.001 ,
    restore_best_weights=True # Восстановить веса модели с наилучшей метрикой
)

### Обучим модель

In [ ]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.7557 - loss: 0.5165 - val_accuracy: 0.9047 - val_loss: 0.2479
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9216 - loss: 0.2030 - val_accuracy: 0.9327 - val_loss: 0.1847
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9529 - loss: 0.1342 - val_accuracy: 0.9496 - val_loss: 0.1428
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9675 - loss: 0.1021 - val_accuracy: 0.9406 - val_loss: 0.1652
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9661 - loss: 0.0996 - val_accuracy: 0.9025 - val_loss: 0.2587
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9681 - loss: 0.0832 - val_accuracy: 0.9159 - val_loss: 0.2170
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9815 - loss: 0.0565 - val_accuracy: 0.9507 - val_loss: 0.1327
Epoch 8/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9908 - loss: 0.0344 - val_accuracy: 0.9395 - val_loss:

KeyboardInterrupt: 

: 

### Посмотрим что там по графикам обучения

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
# Обновляем названия метрик для бинарной классификации
axs[0].plot(np.arange(1, EPOCHS + 1), history.history['loss'][:], label='Функция потерь обучения')
axs[0].plot(np.arange(1, EPOCHS + 1), history.history['val_loss'][:], label='Функция потерь валидации')
axs[0].grid()
axs[0].set_xticks(np.arange(0, EPOCHS + 1, 2))
axs[0].set_xticks(np.arange(0, EPOCHS + 1, 1), minor=True)
axs[0].legend(prop={'size': 10})
axs[0].set_xlabel('Эпоха')
axs[0].set_ylabel('Функция потерь')

axs[1].plot(np.arange(1, EPOCHS + 1), history.history['accuracy'][:], label='Точность обучения')
axs[1].plot(np.arange(1, EPOCHS + 1), history.history['val_accuracy'][:], label='Точность валидации')
axs[1].grid()
axs[1].set_xticks(np.arange(0, EPOCHS + 1, 2))
axs[1].set_xticks(np.arange(0, EPOCHS + 1, 1), minor=True)
axs[1].legend(prop={'size': 10})
axs[1].set_xlabel('Эпоха')
axs[1].set_ylabel('Метрика accuracy')
plt.tight_layout()
plt.show()

### Потестируем модель

In [ ]:
test_loss, test_accuracy = model.evaluate(val_ds, verbose=2)
print(f"\nТочность на валидационном наборе: {test_accuracy*100:.2f}%")
print(f"Функция потерь на валидационном наборе: {test_loss:.4f}")

# Получение меток классов из датасета
class_names = val_ds.class_names
print(f"Названия классов: {class_names}")

# Получение одного батча из валидационного датасета для визуализации
for images, labels in val_ds.take(1):
    predictions = model.predict(images)
    # Преобразуем предсказания из сигмоиды в бинарный класс (0 или 1)
    predicted_classes = (predictions > 0.5).astype(int).flatten()
    labels = labels.numpy().astype(int)

    # Визуализация первых 9 изображений из батча
    plt.figure(figsize=(10, 7))
    for i in range(min(9, len(images))):  # Не превышать размер батча
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'Истинный: {class_names[labels[i]]}\nПредсказанный: {class_names[predicted_classes[i]]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
